# Setup our vulnerability data science lab environment

First we'll import all the libraries we need. A couple of them need installed first. JQ is a pythonic implementation of jq; a tool for querying json really fast. When looking at 25 years of vulnerabilities it is enormously useful.

In [1]:
!pip install jq==1.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 65.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install tqdm==4.67.1


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Let's run the imports
import requests
import pandas as pd
from tqdm import tqdm
import os
import jq
import numpy as np
import re
from pandas.plotting import autocorrelation_plot
import time
import requests
import os
from datetime import datetime
import json 

The folders where we will store the data as json files need to be created. We store 2000 vulnerabilities per file for simpliciity. They are indexed according to their numbers.

In [4]:
file_exists = os.path.exists('CVE-NVD')
if not file_exists:
  os.mkdir('CVE-NVD')
  os.mkdir('CVE-NVD/JSON')

Now we'll download all the NVD data since 1999 using their API. You will need to provide your own API key, which you can get from NVD itself. 
PROTIP: The progress bar comes for free from the tqdm package. Just wrap a for loop in tqdm.tqdm() it and you get a progress bar for free. Now after this tutorial if you keep this notebook, you'll always be able to fetch all this CVE data easily. Handy for many more things than just forecasting.
NOTE: We need to merge these data structures, as we may have updates for some vulnerabilities.

In [5]:
import time
import requests
import os
import tqdm

# Placeholder for your API key from NVD
API_KEY = ""

# Base URL for the NVD API
BASE_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# Create directories if they don't exist
file_exists = os.path.exists('CVE-NVD')
if not file_exists:
    os.mkdir('CVE-NVD')
    os.mkdir('CVE-NVD/JSON')

# Rate limit: 50 requests per 30 seconds
RATE_LIMIT = 50
RATE_LIMIT_WINDOW = 30  # seconds

# Counter for requests
request_count = 0
start_time = time.time()

# Pagination parameters
start_index = 0
results_per_page = 2000  # Maximum allowed by the API

while True:
    params = {
        "startIndex": start_index,
        "resultsPerPage": results_per_page,
    }
    headers = {'apiKey': API_KEY}

    response = requests.get(BASE_URL, params=params, headers=headers)

    # Rate limiting logic
    request_count += 1
    if request_count >= RATE_LIMIT:
        elapsed_time = time.time() - start_time
        if elapsed_time < RATE_LIMIT_WINDOW:
            time.sleep(RATE_LIMIT_WINDOW - elapsed_time)
        request_count = 0
        start_time = time.time()

    if response.status_code == 200:
        data = response.json()
        total_results = data.get("totalResults", 0)

        # Save the current page of results
        with open(f'CVE-NVD/JSON/cve_data_{start_index}.json', 'w') as f:
            f.write(response.text)

        # Check if we have fetched all results
        if start_index + results_per_page >= total_results:
            print("All data has been fetched succesfully.")
            break

        # Update the start index for the next page
        start_index += results_per_page
    elif response.status_code == 522:
        print('Network issues trying this request again.')
        response = requests.get(BASE_URL, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            total_results = data.get("totalResults", 0)

            # Save the current page of results
            with open(f'CVE-NVD/JSON/cve_data_{start_index}.json', 'w') as f:
                f.write(response.text)
        else:
            print("Two network failures in a row, quitting datafetch. Please re-run the code later.")
            break
    elif response.status_code == 401:
        print('Check your API key')
        break
    else:
        print(f"Failed to fetch data: {response.status_code}")
        break

All data has been fetched succesfully.


# Convert the data to panda dataframes and csv files

Here we start to use JQ to make queiries specific to CVE json structure. We pull out the CVE-ID, the published date, the assigner, and the CVSSv2 base score, as well as other details and push them into a pandas dataframe. This dataframe can then be saved and reused regularly.

In [6]:
# Combined jq query to extract all relevant vulnerability data
vuln_query = jq.compile("""
  .vulnerabilities[] | {
    "ID": .cve.id,
    "Publication": .cve.published,
    "ASSIGNER": .cve.sourceIdentifier,
    "DESCRIPTION": [.cve.descriptions[].value],
    "v2 CVSS": (if .cve.metrics.cvssMetricV2 and (.cve.metrics.cvssMetricV2 | length > 0) 
                     then .cve.metrics.cvssMetricV2[0].cvssData.baseScore 
                     else null end),
    "v2 Exploitability Score": (if .cve.metrics.cvssMetricV2 and (.cve.metrics.cvssMetricV2 | length > 0) 
                                    then .cve.metrics.cvssMetricV2[0].exploitabilityScore 
                                    else null end),
    "v2 Vector": (if .cve.metrics.cvssMetricV2 and (.cve.metrics.cvssMetricV2 | length > 0) 
                     then .cve.metrics.cvssMetricV2[0].cvssData.vectorString 
                     else null end),
    "v3 CVSS": (if .cve.metrics.cvssMetricV31 and (.cve.metrics.cvssMetricV31 | length > 0) 
                     then .cve.metrics.cvssMetricV31[0].cvssData.baseScore
                     elif .cve.metrics.cvssMetricV30 and (.cve.metrics.cvssMetricV30 | length > 0) 
                     then .cve.metrics.cvssMetricV30[0].cvssData.baseScore 
                     else null end),
    "v3 Vector": (if .cve.metrics.cvssMetricV31 and (.cve.metrics.cvssMetricV31 | length > 0) 
                     then .cve.metrics.cvssMetricV31[0].cvssData.vectorString 
                     elif .cve.metrics.cvssMetricV30 and (.cve.metrics.cvssMetricV30 | length > 0) 
                     then .cve.metrics.cvssMetricV30[0].cvssData.vectorString 
                     else null end),
    "v3 Exploitability Score": (if .cve.metrics.cvssMetricV31 and (.cve.metrics.cvssMetricV31 | length > 0) 
                                    then .cve.metrics.cvssMetricV31[0].exploitabilityScore 
                                    elif .cve.metrics.cvssMetricV30 and (.cve.metrics.cvssMetricV30 | length > 0) 
                                    then .cve.metrics.cvssMetricV30[0].exploitabilityScore 
                                    else null end),
    "v2.3 CPE": [.cve.configurations[]?.nodes[].cpeMatch[]? | select(.vulnerable == true) | .criteria] // [],
    "CWE": [.cve.weaknesses[]?.description[].value],
    "VulnStatus": .cve.vulnStatus
  }
""")

# Function to process a single file and extract vulnerabilities
def process_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)  # Load the JSON data from the file
    
    # Apply the jq query to extract vulnerabilities
    vuln_data = vuln_query.input(data).all()  # List of dictionaries for each vulnerability
    
    return vuln_data

# Function to process multiple files in a directory with progress bar
def process_directory(directory_path):
    all_vulns = []  # List to hold vulnerabilities from all files
    json_files = [f for f in os.listdir(directory_path) if f.endswith('.json')]  # Filter JSON files
    
    # Use tqdm to create a progress bar for file processing
    for filename in tqdm.tqdm(json_files, desc="Processing Files", unit="file"):
        file_path = os.path.join(directory_path, filename)
        
        # Process each file
        vuln_data = process_file(file_path)
        all_vulns.extend(vuln_data)  # Append the extracted data from this file
    
    # Return a list of all vulnerabilities found
    return all_vulns

# Define the directory where your JSON files are stored
json_dir = 'CVE-NVD/JSON/'

# Process all JSON files in the directory
vulnerabilities = process_directory(json_dir)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(vulnerabilities)

# Optional: Clean up list-based fields (like 'description', 'cpe_criteria', 'cwe')
df['DESCRIPTION'] = df['DESCRIPTION'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
#df['v2.3 CPE'] = df['v2.3 CPE'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
df['CWE'] = df['CWE'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
# Add a 'Count' column with all values set to 1 (syntactic sugar to make counts and sums and forecasting easy)
df['Count'] = 1

# Show the last few rows of the DataFrame
print(df.tail)

Processing Files: 100%|██████████| 156/156 [02:08<00:00,  1.22file/s]
<bound method NDFrame.tail of                    ID              Publication                ASSIGNER  \
0       CVE-1999-0095  1988-10-01T04:00:00.000           cve@mitre.org   
1       CVE-1999-0082  1988-11-11T05:00:00.000           cve@mitre.org   
2       CVE-1999-1471  1989-01-01T05:00:00.000           cve@mitre.org   
3       CVE-1999-1122  1989-07-26T04:00:00.000           cve@mitre.org   
4       CVE-1999-1467  1989-10-26T04:00:00.000           cve@mitre.org   
...               ...                      ...                     ...   
311902  CVE-2018-2591  2018-01-18T02:29:18.757  secalert_us@oracle.com   
311903  CVE-2018-2592  2018-01-18T02:29:18.787  secalert_us@oracle.com   
311904  CVE-2018-2593  2018-01-18T02:29:18.837  secalert_us@oracle.com   
311905  CVE-2018-2594  2018-01-18T02:29:18.883  secalert_us@oracle.com   
311906  CVE-2018-2595  2018-01-18T02:29:18.930  secalert_us@oracle.com   

           

Save all the data we just filtered to a CSV file, for future use.

In [7]:
df.head()

,ID,Publication,ASSIGNER,DESCRIPTION,v2 CVSS,v2 Exploitability Score,v2 Vector,v3 CVSS,v3 Vector,v3 Exploitability Score,v2.3 CPE,CWE,VulnStatus,Count
0,CVE-1999-0095,1988-10-01T04:00:00.000,cve@mitre.org,"The debug command in Sendmail is enabled, allo...",10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,[cpe:2.3:a:eric_allman:sendmail:5.58:*:*:*:*:*...,NVD-CWE-Other,Deferred,1
1,CVE-1999-0082,1988-11-11T05:00:00.000,cve@mitre.org,CWD ~root command in ftpd allows root access.,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:a:ftp:ftp:*:*:*:*:*:*:*:*, cpe:2.3:a:...",NVD-CWE-Other,Deferred,1
2,CVE-1999-1471,1989-01-01T05:00:00.000,cve@mitre.org,Buffer overflow in passwd in BSD based operati...,7.2,3.9,AV:L/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:o:bsd:bsd:4.2:*:*:*:*:*:*:*, cpe:2.3:...",NVD-CWE-Other,Deferred,1
3,CVE-1999-1122,1989-07-26T04:00:00.000,cve@mitre.org,Vulnerability in restore in SunOS 4.0.3 and ea...,4.6,3.9,AV:L/AC:L/Au:N/C:P/I:P/A:P,NaN,None,NaN,"[cpe:2.3:o:sun:sunos:*:*:*:*:*:*:*:*, cpe:2.3:...",NVD-CWE-Other,Deferred,1
4,CVE-1999-1467,1989-10-26T04:00:00.000,cve@mitre.org,Vulnerability in rcp on SunOS 4.0.x allows rem...,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:o:sun:sunos:4.0:*:*:*:*:*:*:*, cpe:2....",NVD-CWE-Other,Deferred,1


In [8]:
# Check if the directory for the CSV file exists, and create it if necessary
csv_file_path = 'NVD-Vulnerability-Volumes.csv'

# Check if the file already exists
if os.path.exists(csv_file_path):
    # If the file exists, read it into a DataFrame
    existing_data = pd.read_csv(csv_file_path, index_col='ID')

    # Merge the existing data with the new data
    all_items = pd.concat([existing_data, df.set_index('ID')])

    # Drop duplicate rows based on the 'cve_id' column, keeping the latest entry
    all_items = all_items[~all_items.index.duplicated(keep='last')]

# Reset the index to publication after dedupping based on IDs
all_items = df.set_index('Publication')

# Sort the data by the index (published date)
all_items.sort_index(inplace=True)

# Save the merged data back to the CSV file
all_items.to_csv(csv_file_path)

# Now we want to clone this data frame and explode the cpe column so we have a ready made dataframe that can do vendor and product forecasting

In [9]:
all_items

,ID,ASSIGNER,DESCRIPTION,v2 CVSS,v2 Exploitability Score,v2 Vector,v3 CVSS,v3 Vector,v3 Exploitability Score,v2.3 CPE,CWE,VulnStatus,Count
Publication,,,,,,,,,,,,,
1988-10-01T04:00:00.000,CVE-1999-0095,cve@mitre.org,"The debug command in Sendmail is enabled, allo...",10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,[cpe:2.3:a:eric_allman:sendmail:5.58:*:*:*:*:*...,NVD-CWE-Other,Deferred,1
1988-11-11T05:00:00.000,CVE-1999-0082,cve@mitre.org,CWD ~root command in ftpd allows root access.,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:a:ftp:ftp:*:*:*:*:*:*:*:*, cpe:2.3:a:...",NVD-CWE-Other,Deferred,1
1989-01-01T05:00:00.000,CVE-1999-1471,cve@mitre.org,Buffer overflow in passwd in BSD based operati...,7.2,3.9,AV:L/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:o:bsd:bsd:4.2:*:*:*:*:*:*:*, cpe:2.3:...",NVD-CWE-Other,Deferred,1
1989-07-26T04:00:00.000,CVE-1999-1122,cve@mitre.org,Vulnerability in restore in SunOS 4.0.3 and ea...,4.6,3.9,AV:L/AC:L/Au:N/C:P/I:P/A:P,NaN,None,NaN,"[cpe:2.3:o:sun:sunos:*:*:*:*:*:*:*:*, cpe:2.3:...",NVD-CWE-Other,Deferred,1
1989-10-26T04:00:00.000,CVE-1999-1467,cve@mitre.org,Vulnerability in rcp on SunOS 4.0.x allows rem...,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,"[cpe:2.3:o:sun:sunos:4.0:*:*:*:*:*:*:*, cpe:2....",NVD-CWE-Other,Deferred,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-29T01:15:35.847,CVE-2025-9904,f98c90f0-e9bd-4fa7-911b-51993f3571fd,Unallocated memory access vulnerability in pri...,NaN,NaN,None,5.3,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:L,3.9,[],CWE-696,Received,1
2025-09-29T02:15:31.473,CVE-2025-11135,cna@vuldb.com,A vulnerability was detected in pmTicket Proje...,7.5,10.0,AV:N/AC:L/Au:N/C:P/I:P/A:P,7.3,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:L/I:L/A:L,3.9,[],"CWE-20, CWE-502",Received,1
2025-09-29T03:15:42.063,CVE-2025-11136,cna@vuldb.com,A flaw has been found in YiFang CMS up to 2.0....,5.8,6.4,AV:N/AC:L/Au:M/C:P/I:P/A:P,4.7,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:L/I:L/A:L,1.2,[],"CWE-284, CWE-434",Received,1


In [10]:
def process_cpe_dataframe(df):
    # Explode the 'v2.3 CPE' column to create a new row for each CPE string
    df = df.explode('v2.3 CPE')
    
    def extract_cpe_parts(cpe_str):
        if pd.isna(cpe_str) or not isinstance(cpe_str, str):
            return pd.Series({
                'Part': None, 'Vendor': None, 'Product': None,
                'Version': None, 'Update': None, 'Edition': None,
                'Language': None, 'SW_Edition': None, 'Target_SW': None,
                'Target_HW': None, 'Other': None
            })
        
        cpe_str = cpe_str.strip('"')
        parts = cpe_str.split(':')
        
        # Ensure we have enough parts
        if len(parts) >= 13:
            return pd.Series({
                'Part': parts[2],
                'Vendor': parts[3],
                'Product': parts[4],
                'Version': parts[5],
                'Update': parts[6],
                'Edition': parts[7],
                'Language': parts[8],
                'SW_Edition': parts[9],
                'Target_SW': parts[10],
                'Target_HW': parts[11],
                'Other': parts[12] if len(parts) > 12 else None
            })
        
        return pd.Series({
            'Part': None, 'Vendor': None, 'Product': None,
            'Version': None, 'Update': None, 'Edition': None,
            'Language': None, 'SW_Edition': None, 'Target_SW': None,
            'Target_HW': None, 'Other': None
        })
    
    # Apply the extraction function to each row in the DataFrame
    extracted_parts = df['v2.3 CPE'].apply(lambda x: extract_cpe_parts(x))
    
    # Concatenate the original DataFrame with the extracted parts
    df = pd.concat([df, extracted_parts], axis=1)
    
    return df

In [11]:
# Re-run the function with the corrected implementation
cpe_df = process_cpe_dataframe(all_items)

In [12]:
cpe_df.head()

,ID,ASSIGNER,DESCRIPTION,v2 CVSS,v2 Exploitability Score,v2 Vector,v3 CVSS,v3 Vector,v3 Exploitability Score,v2.3 CPE,...,Vendor,Product,Version,Update,Edition,Language,SW_Edition,Target_SW,Target_HW,Other
Publication,,,,,,,,,,,,,,,,,,,,,
1988-10-01T04:00:00.000,CVE-1999-0095,cve@mitre.org,"The debug command in Sendmail is enabled, allo...",10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,cpe:2.3:a:eric_allman:sendmail:5.58:*:*:*:*:*:*:*,...,eric_allman,sendmail,5.58,*,*,*,*,*,*,*
1988-11-11T05:00:00.000,CVE-1999-0082,cve@mitre.org,CWD ~root command in ftpd allows root access.,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,cpe:2.3:a:ftp:ftp:*:*:*:*:*:*:*:*,...,ftp,ftp,*,*,*,*,*,*,*,*
1988-11-11T05:00:00.000,CVE-1999-0082,cve@mitre.org,CWD ~root command in ftpd allows root access.,10.0,10.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,cpe:2.3:a:ftpcd:ftpcd:*:*:*:*:*:*:*:*,...,ftpcd,ftpcd,*,*,*,*,*,*,*,*
1989-01-01T05:00:00.000,CVE-1999-1471,cve@mitre.org,Buffer overflow in passwd in BSD based operati...,7.2,3.9,AV:L/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,cpe:2.3:o:bsd:bsd:4.2:*:*:*:*:*:*:*,...,bsd,bsd,4.2,*,*,*,*,*,*,*
1989-01-01T05:00:00.000,CVE-1999-1471,cve@mitre.org,Buffer overflow in passwd in BSD based operati...,7.2,3.9,AV:L/AC:L/Au:N/C:C/I:C/A:C,NaN,None,NaN,cpe:2.3:o:bsd:bsd:4.3:*:*:*:*:*:*:*,...,bsd,bsd,4.3,*,*,*,*,*,*,*


In [13]:
# Remove rows where 'v2.3 CPE' column is NaN
cpe_df = cpe_df.dropna(subset=['v2.3 CPE'])

# Remove rows where 'VulnStatus' column is 'Rejected'
cpe_df = cpe_df[cpe_df['VulnStatus'] != 'Rejected']

# Reset the index to make 'Publication' a column
cpe_df.reset_index(inplace=True)

# Set a multi-index with 'ID' and 'v2.3 CPE'
cpe_df.set_index(['ID', 'v2.3 CPE'], inplace=True)

In [14]:
# Check if the file already exists
csv_file_path = 'Vendor-Product-Vulnerability-Volumes.csv'
if os.path.exists(csv_file_path):
    # If the file exists, read it into a DataFrame
    existing_data = pd.read_csv(csv_file_path, low_memory=False)

    # Merge the existing data with the new data on ID and CPE
    merged_cpe_df = pd.concat([existing_data, cpe_df], ignore_index=True)

    # Ensure uniqueness by considering both ID and CPE columns
    merged_cpe_df = merged_cpe_df.drop_duplicates(subset=['ID', 'v2.3 CPE'], keep='last')
else:
    # If the file doesn't exist, use the new data as is
    merged_cpe_df = cpe_df.copy()

# Sort the data by the ID column
merged_cpe_df.sort_values(by='ID', inplace=True)

# Save the merged data back to the CSV file
merged_cpe_df.to_csv(csv_file_path, index=True)

If you want to read that file in the future, without fetching all the data again, just uncoment the cell below.

In [15]:
#all_items = pd.read_csv('NVD-Vulnerability-Volumes.csv',index_col=['published'],parse_dates=['published'], low_memory=False)
#all_items = all_items.sort_index()

Now we create a vulnlambda file as well, based of the data we already hold.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7acd54e3-f1e9-4bb5-a625-0a781a5b944c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>